In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download de_core_news_sm
!pip install emoji
!pip install langdetect
!pip install transformers
!pip install ekphrasis
!pip install -U datasets
!pip install pymongo_ssh

There are some helper functions written in some external python files. We load these python files directly into colab.

In [1]:
from google.colab import files
files.upload()

Saving inferencing.py to inferencing.py
Saving model_params.py to model_params.py
Saving preprocessing.py to preprocessing.py
Saving training.py to training.py


{'inferencing.py': b'import hashlib \nimport pymongo\nimport torch\nimport math\n\nfrom pymongo_ssh import MongoSession\n\n\n# connect to database\ndef connect_to_database(key, host=\'138.246.233.159\'):\n    session = MongoSession(\n    host=host,\n    port=22,\n    user=\'ubuntu\',\n    key=key,\n    uri=\'mongodb://127.0.0.1:27017\')\n    db = session.connection[\'telegram\']\n    \n    db_channels = db.channels\n    db_messages = db.messages\n    db_errors = db.errors\n    db_users = db.users\n    db_entities = db.entities\n    \n    return db_channels, db_messages, db_errors, db_users, db_entities\n\ndef hashContent(field1,field2,field3):\n    to_hash = field1 if field1 is not None else \'\'\n    if field2 is not None:\n        to_hash = to_hash + field2\n    if field3 is not None:\n        to_hash = to_hash + field3\n    result = hashlib.md5(to_hash.encode()) \n    return result.hexdigest()\n\ndef predict(text,tokenizer,model):\n    inputs =  tokenizer(text, return_tensors="pt", 

The models are permanently stored on our Google Drive accounts. So we mount drive in order to access the models.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import multiprocessing
import pymongo
import pandas as pd
import spacy
import torch
import operator
import hashlib 
import numpy as np
import multiprocessing
import nltk
nltk.download('stopwords')
import preprocessing as pp
import inferencing as ig
import model_params as mp
import time
import sys
import random
import gc

from transformers import AutoModel, AutoTokenizer, Trainer, TrainingArguments,BertForSequenceClassification
from bson.objectid import ObjectId
from pymongo import MongoClient
from pymongo import UpdateOne
from tqdm import tqdm
from spacy.language import Language
from pandas import Panel

tqdm.pandas()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
db_channels, db_messages, db_errors, db_users, db_entities = ig.connect_to_database(key='/content/drive/My Drive/NLP/id_hsd')

In [5]:
selected_dataset = 'hasoc_2020_task_2'
MODEL_NAME = 'dbmdz/bert-base-german-cased'
#MODEL_NAME = 'german-nlp-group/electra-base-german-uncased'
#MODEL_NAME = 'deepset/gbert-base'
max_length = 412
workers = 5

Both Hasoc models have the following mapping 'NONE' -> 3, 'PRFN' -> 1, 'OFFN' -> 2, 'HATE' -> 0. So it does not follow the rule: "The more offensive, the higher the index". To fix this we have to map 'NONE' -> 0 and 'HATE' -> 3. Otherwise the inference would not work properly for long messages that have to be split.

In [6]:
def hasoc_multiclass_map(idx):
  if idx == 0:
    return 3
  if idx == 3:
    return 0
  return idx

In [7]:
def inferenceMessages(df, queue_done, selected_dataset, lookup_table_hash_object_ids):
    time.sleep(random.randint(0,30))
    print("Worker has started with", len(df))
    # connect to database
    db_channels, db_messages, db_errors, db_users, db_entities = ig.connect_to_database(key='/content/drive/My Drive/NLP/id_hsd')

    # load classificaton model
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = BertForSequenceClassification.from_pretrained(f"/content/drive/My Drive/NLP/{selected_dataset}")
    model = model.cuda()
    print("Model is loaded")
    
    # sentence splitter
    nlp = spacy.load("de_core_news_sm")
    @Language.component("set_custom_boundaries")
    def set_custom_boundaries(doc):
        for token in doc[:-1]:
            if token.text == "...":
                doc[token.i + 1].is_sent_start = True
        return doc
    nlp.add_pipe("set_custom_boundaries", before="parser")
    
    
    field_name = f"hate_{selected_dataset}"
    
    bulk = []
    for index, row in df.iterrows():
        combined_text = row['text']
        if row['link_preview_title'] is not None:
            combined_text = combined_text + ". " +  row['link_preview_title']
        if row['link_preview'] is not None:
            combined_text = combined_text + ". " +  row['link_preview']

        combined_text = combined_text.replace('. . ','. ')

        if combined_text is None:
            continue

        text = pp.prepare_single_text(combined_text)

        if len(text.split(' ')) <= max_length:
            # the text doesn't need to be splitted
            # 412 is to make sure that we have fewer than 512 tokens
            index, value = max(enumerate(ig.predict(text,tokenizer,model)), key=operator.itemgetter(1))
            label = mp.model_params[selected_dataset]['mapping_reverse'][index]
            confidence = value

        else:
            # text has too many tokens
            # it has to be split
            doc = nlp(combined_text)
            sentences = [sent.text for sent in doc.sents]
            texts = ['']
            labels_confidences = []
            pointer = 0
            # create sentence buckets
            for sentence in sentences:
                try:
                    text_preprocessed = pp.prepare_single_text(texts[pointer] + " " + sentence.strip())
                except Exception as e:
                    print(e)
                    print(str(sentence.text.strip()))
                if len(text_preprocessed.split(' ')) <= max_length:
                    texts[pointer] = texts[pointer] + " " + sentence.strip()
                else:
                    pointer = pointer + 1
                    texts.append(sentence.strip())

            
            # predict each bucket
            for bucket in texts:
                index, confidence = max(enumerate(ig.predict(bucket,tokenizer,model)), key=operator.itemgetter(1))
                label_idx = index

                # As stated above, for the multiclass hasoc models we have to fix the mapping
                if selected_dataset in ['hasoc_2020_task_2', 'hasoc_1920_task_2']:
                  label_idx = hasoc_multiclass_map(label_idx)
                labels_confidences.append([label_idx, confidence])

            max_label_idx = -1
            confidence = 0

            # prefer the bucket that is classified as the most offensive
            # the higher the label_idx, the more offensive is the message
            # as the confidence we keep the highest confidece of the most offensive label
            for prediction in labels_confidences:
              label_idx = prediction[0]
              if label_idx == max_label_idx:
                confidence = max(confidence,prediction[1])
              elif label_idx > max_label_idx:
                max_label_idx = label_idx
                confidence = prediction[1]

            # now we have to "undo" the mapping (reverse the mapping)
            if selected_dataset in ['hasoc_2020_task_2', 'hasoc_1920_task_2']:
                  max_label_idx = hasoc_multiclass_map(max_label_idx)
            label = mp.model_params[selected_dataset]['mapping_reverse'][max_label_idx]

        # create update object
        for objectid_to_update in lookup_table_hash_object_ids[row['hash']]:
            update_query = UpdateOne({'_id': objectid_to_update},{'$set':{field_name:{'label':label,'confidence':confidence.item()}}}, upsert=True)
            queue_done.put(1)
            bulk.append(update_query)

        # save to database
        if len(bulk) >= 2000:
            try:
                result = db_messages.bulk_write(bulk)
                bulk = []
            except Exception as e:
                print(e)
                print(bulk)  

    # save the rest to database
    try:
        _ = db_messages.bulk_write(bulk)
        bulk = []
    except Exception as e:
        print(e)
        print(bulk) 

In [ ]:
if __name__ == '__main__':
    field_name = f"hate_{selected_dataset}"
    query={'language':'de', field_name : {'$exists': False}}

    # get the messages from the database
    df_messages = pd.DataFrame(list(db_messages.find(query,{'channel_name':1,'text':1,'link_preview_title':1,'link_preview':1})))
    print("Number of all messages:",len(df_messages))
    time.sleep(1)
    
    # hash content for deduplicating
    df_messages['hash'] = df_messages.progress_apply(lambda x: ig.hashContent(x['text'],x['link_preview_title'],x['link_preview']), axis=1)

    # find distinct messages
    df_messages_distinct = df_messages.drop_duplicates(subset=['hash'],keep='first')
    print("Number of distinc messages:", len(df_messages_distinct))
    time.sleep(1)

    # create lookup table
    lookup_table_hash_object_ids = { i : [] for i in df_messages_distinct['hash'].to_list() }
    for index,row in tqdm(df_messages.iterrows(), total=df_messages.shape[0]):
        lookup_table_hash_object_ids[row['hash']].append(row['_id'])
    
    time.sleep(2) 
    # split dataframe into n chunks / n == number of workers
    # one chunk for each worker
    dfs = np.array_split(df_messages_distinct, workers)
    
    total = len(df_messages_distinct)
    del [[df_messages,df_messages_distinct]]
    gc.collect()
    
    df_messages = pd.DataFrame()
    df_messages_distinct = pd.DataFrame()
    
    queue_done = multiprocessing.JoinableQueue()
    
    cur = 0

    with tqdm(total=total, file=sys.stdout) as pbar:
        jobs = []
        # worker processes
        for i in range(workers):
            p = multiprocessing.Process(target=inferenceMessages, 
                                        args=(dfs[i],
                                              queue_done,
                                              selected_dataset,
                                              lookup_table_hash_object_ids,
                                             ))
            p.daemon = True
            p.start()
            jobs.append(p)

        while cur < total:
            update = 0
            while not queue_done.empty():
                elem = queue_done.get()
                update += elem
            pbar.update(update)
            cur += update
            time.sleep(2)               
        # update to 100%
        pbar.update(total-cur)                       
        print('Progress bar completed.')
        # wait until all jobs are done
        for job in jobs:
            job.join()